In [1]:
from jina import Flow, Executor, requests, Document, DocumentArray
import pandas as pd
import numpy as np 

### 3 main fundamental jina concepts 

* document
* executor 
* flow

In [2]:
df = pd.read_csv('lyrics-JINA - Sheet1.csv')
df.head()

,Index,Singer,Song,Lyrics,Collected By
0,1,Bruno Mars,24 K Magic,Tonight\nI just want to take you higher\nThrow...,Ishita
1,2,Taylor Swift,Wildest Dreams,"He said, ""Let's get out of this town\nDrive ou...",Ishita
2,3,Green Day,Numb,I'm tired of being what you want me to be\nFee...,Ishita
3,4,Macklemore,Glorius,You know I'm back like I never left\nAnother s...,Ishita
4,5,Ariana Grande,POV,It's like you got superpowers\nTurn my minutes...,Ishita


In [3]:
df = df.drop_duplicates().dropna()
df.shape

(33, 5)

### sample songs

In [23]:
for ind in range(3):
    print()
    print(f" Song name : {df.iloc[ind, 2]} ".center(80, ' '))
    print(f" Singer Name : {df.iloc[ind, 1]} ".center(80, ' '))
    # print(f" Singer name : {df.iloc[ind, 0]} ".center(80, ' '))
    print()
    print(df.iloc[ind, 3])
    print()
    print('-' * 80)


                             Song name : 24 K Magic                             
                            Singer Name : Bruno Mars                            

Tonight
I just want to take you higher
Throw your hands up in the sky
Let's set this party off right
Players, put yo' pinky rings up to the moon
Girls, what y'all trying to do?
Twenty four karat magic in the air
Head to toe so player
Look out uh
Pop pop, it's show time (show time)
Show time (show time)
Guess who's back again?
Oh they don't know? (Go on tell 'em)
Oh they don't know? (Go on tell 'em)
I bet they know soon as we walk in (showin' up)
Wearing Cuban links (ya)
Designer minks (ya)
Inglewood's finest shoes (whoop, whoop)
Don't look too hard might hurt ya'self
Known to give the color red the blues
Oh shit, I'm a dangerous man with some money in my pocket (keep up)
So many pretty girls around me and they waking up the rocket (keep up)
Why you mad, fix ya face, ain't my fault y'all be jocking (keep up)
Players only, com

### creating a document and documentArray

In [5]:
# basic document 

# Document(text = 'something')  # use it for strings 
# Document(content = 'something') # use it when you dont know what type of jina data type it is 
# Document(blob = 'img, vid, audio') # use it for images, audios and videos
# Document(uri = 'local file path or url link') # can be any form a local or remote link

In [24]:
docs = DocumentArray()
for ind in range(df.shape[0]):
    song_name = df.iloc[ind, 2]
    lyr = df.iloc[ind, 3]
    doc = Document(text = lyr)
    doc.tags['song_name'] = song_name
    docs.append(doc)
    
docs

<jina.types.arrays.document.DocumentArray length=33 at 140042043757520>

In [25]:
docs[0].json()

'{\n  "id": "8ac44a3e-3cb2-11ec-b50c-b56bdb58f59d",\n  "mime_type": "text/plain",\n  "tags": {\n    "song_name": "24 K Magic"\n  },\n  "text": "Tonight\\nI just want to take you higher\\nThrow your hands up in the sky\\nLet\'s set this party off right\\nPlayers, put yo\' pinky rings up to the moon\\nGirls, what y\'all trying to do?\\nTwenty four karat magic in the air\\nHead to toe so player\\nLook out uh\\nPop pop, it\'s show time (show time)\\nShow time (show time)\\nGuess who\'s back again?\\nOh they don\'t know? (Go on tell \'em)\\nOh they don\'t know? (Go on tell \'em)\\nI bet they know soon as we walk in (showin\' up)\\nWearing Cuban links (ya)\\nDesigner minks (ya)\\nInglewood\'s finest shoes (whoop, whoop)\\nDon\'t look too hard might hurt ya\'self\\nKnown to give the color red the blues\\nOh shit, I\'m a dangerous man with some money in my pocket (keep up)\\nSo many pretty girls around me and they waking up the rocket (keep up)\\nWhy you mad, fix ya face, ain\'t my fault y\'al

In [26]:
docs[0].text

"Tonight\nI just want to take you higher\nThrow your hands up in the sky\nLet's set this party off right\nPlayers, put yo' pinky rings up to the moon\nGirls, what y'all trying to do?\nTwenty four karat magic in the air\nHead to toe so player\nLook out uh\nPop pop, it's show time (show time)\nShow time (show time)\nGuess who's back again?\nOh they don't know? (Go on tell 'em)\nOh they don't know? (Go on tell 'em)\nI bet they know soon as we walk in (showin' up)\nWearing Cuban links (ya)\nDesigner minks (ya)\nInglewood's finest shoes (whoop, whoop)\nDon't look too hard might hurt ya'self\nKnown to give the color red the blues\nOh shit, I'm a dangerous man with some money in my pocket (keep up)\nSo many pretty girls around me and they waking up the rocket (keep up)\nWhy you mad, fix ya face, ain't my fault y'all be jocking (keep up)\nPlayers only, come on\nPut your pinky rings up to the moon\nGirls, what y'all trying to do?\nTwenty four karat magic in the air\nHead to toe so player\nUh, l

In [27]:
docs[0]

### creating your own executor

In [28]:
class Clean(Executor):
    @requests
    def clean(self, docs, parameters, **kwargs):
        print(parameters)
        return DocumentArray(list(filter(lambda x : len(x.text) > 0, docs)))

### Flow

In [29]:
model = "sentence-transformers/paraphrase-distilroberta-base-v1"

flow = (
    Flow()
    .add(
        name="lyrics_text_encoder",
        uses="jinahub://TransformerTorchEncoder",
        uses_with={"pretrained_model_name_or_path": model},
    )
    .add(
        name="lyrics_text_indexer",
        uses='jinahub://SimpleIndexer',
    )
)

In [30]:
flow

In [33]:
#docs[0]

In [36]:
!rm -rf workspace # Remove workspace in case we've indexed stuff before

In [37]:
with flow:
    flow.index(
        inputs=docs,
  )

2021-11-03 20:01:51.719869: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-03 20:01:51.719911: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


lyrics_text_encoder@27640[L]:ready and listening
lyrics_text_indexer@27640[L]:ready and listening
        gateway@27640[L]:ready and listening
           Flow@27640[I]:🎉 Flow is ready to use!
	🔗 Protocol: 		GRPC
	🏠 Local access:	0.0.0.0:59605
	🔒 Private network:	192.168.43.107:59605
lyrics_text_encoder@32135[E]:AttributeError("'DocumentArray' object has no attribute 'batch'")
 add "--quiet-error" to suppress the exception details
Traceback (most recent call last):
  File "/home/sanjju/miniconda3/envs/ml38/lib/python3.8/site-packages/jina/peapods/runtimes/zmq/zed.py", line 250, in _msg_callback
    processed_msg = self._callback(msg)
  File "/home/sanjju/miniconda3/envs/ml38/lib/python3.8/site-packages/jina/peapods/runtimes/zmq/zed.py", line 236, in _callback
    msg = self._post_hook(self._handle(self._pre_hook(msg)))
  File "/home/sanjju/miniconda3/envs/ml38/lib/python3.8/site-packages/jina/peapods/runtimes/zmq/zed.py", line 198, in _handle
    self._data_request_handler.handle(
  Fil

In [38]:
for i in docs:
    if len(i.text) == 0:
        print(i.text)

In [39]:

model = "sentence-transformers/paraphrase-distilroberta-base-v1" # Any model from Huggingface

In [40]:
from jina import Flow

In [ ]:
flow = (
    Flow()
    .add(
        name="error_text_encoder",
        uses="jinahub://TransformerTorchEncoder",
        uses_with={"pretrained_model_name_or_path": model},
    )
    .add(
        name="error_text_indexer",
        uses='jinahub://SimpleIndexer',
    )
)

### indexing 

In [ ]:
!rm -rf workspace

with flow:
    flow.index(
      inputs=docs,
        docs = docs,
        parameters = {'name' : 'somethign', 'xyz' : 'fsdfsdfsa'}
  )

### querying 

In [ ]:
query = Document(text = input('Query product : '))
with flow:
    response = flow.search(inputs = query, return_results = True)

### finding the best match

In [ ]:
matches = response[0].docs[0].matches
matches

In [ ]:
for m in matches:
    print()
    print(f" product name : {m.text} ".center(80, ' '))
    print()
    print(m.tags['description'])
    print()
    print('-' * 80)